In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from collections import defaultdict
import numpy as np 
import os
import re

class Tranf_encode:
    def __init__(self,max_doc_length,min_df):
        self._vocab=[]
        self._max_doc_length=max_doc_length
        self._min_df=min_df
    
    def isappear(self,word,wordslist):
        l=0
        r=len(wordslist)-1
        while(l<=r):
            m=int((r+l)/2)
            if(word==wordslist[m]):
                return m
            else:
                if(word<wordslist[m]):
                    r=m-1
                else:
                    l=m+1
        return -1

    def fit(self,path_train):
        def get_data_and_vocab(path_train):
            data=[]
            vocab_count=defaultdict(int)
            list_folder=os.listdir(path_train)
            list_folder.sort()
            i_folder=0
            for folder in list_folder:
                path_folder=path_train+'/'+folder
                list_doc=os.listdir(path_folder)
                list_doc.sort()
                for doc in list_doc:
                    path_file=path_folder+'/'+doc
                    with open(path_file,'r',errors='ignore') as f:
                        text=f.read()
                    words=re.split('\W+',text)
                    for word in words:
                        vocab_count[word]+=1
                    content=' '.join(words)
                    ifo_doc=str(i_folder)+'<fff>'+str(doc)+'<fff>'+content
                    data.append(ifo_doc)
                i_folder+=1
            
            self._vocab=[word for word in vocab_count.keys()
                                if vocab_count[word]>self._min_df]
            self._vocab.sort()
            return data
        
        data_train=get_data_and_vocab(path_train)
        data_train_embed=self.get_embedding(data_train)
        return data_train_embed
        
    def get_embedding(self,data):
        vocab_id=dict([word, self._vocab.index(word)+2] for word in self._vocab) 
        padding_ID=0
        unknown_ID=1
        data_embed=[]
        for doc in data:
            label,doc_id,text=doc.split('<fff>')
            words=text.split()[:self._max_doc_length]
            doc_embed=[]
            for word in words:
                if self.isappear(word,self._vocab)!=-1:
                    doc_embed.append(str(vocab_id[word]))
                else:
                    doc_embed.append(str(unknown_ID))
            if(len(words)<self._max_doc_length):
                for _ in range(self._max_doc_length-len(words)):
                    doc_embed.append(str(padding_ID))
            data_embed.append(label+'<fff>'+doc_id+'<fff>'+str(len(words))+'<fff>'+' '.join(doc_embed))
        
        return data_embed
        
    def fit_tranf(self,path_in):
        data=[]
        list_folder=os.listdir(path_in)
        list_folder.sort()
        i_folder=0
        for folder in list_folder:
            path_folder=path_in+'/'+folder
            list_doc=os.listdir(path_folder)
            list_doc.sort()
            for doc in list_doc:
                path_doc=path_folder+'/'+doc
                with open(path_doc,'r',errors='ignore') as f:
                    text=f.read()
                words=re.split('\W+',text)
                content=' '.join(words)
                ifo_doc=str(i_folder)+'<fff>'+doc+'<fff>'+content
                data.append(ifo_doc)
            i_folder+=1
        
        data_embed=self.get_embedding(data)
        return data_embed
    
    def get_vocab(self):
        return self._vocab

model=Tranf_encode(500,10)
train=model.fit('/content/drive/My Drive/Datasets/20news-bydate-train')
test=model.fit_tranf('/content/drive/My Drive/Datasets/20news-bydate-test')
vocab=model.get_vocab()

with open('/content/drive/My Drive/Datasets/trainencode.txt','w') as f:
    f.write('\n'.join(train))
with open('/content/drive/My Drive/Datasets/testencode.txt','w') as f:
    f.write('\n'.join(test))
with open('/content/drive/My Drive/Datasets/vocab.txt','w') as f:
    f.write('\n'.join(vocab))


In [0]:
import numpy as np
import random 

class DataReader:
    def __init__(self,path,batch_size):
        with open(path,'r') as f:
            d_line=f.read().splitlines()
        self.x=[]
        self.y=[]
        self.sentence_lengths=[]
        self.batch_size=batch_size
        for d in d_line:
            fea=d.split('<fff>')
            label,sentence_len,data=int(fea[0]),int(fea[2]),fea[-1]
            self.y.append(label)
            self.sentence_lengths.append(sentence_len)
            vector=[int(number) for number in data.split()]
            self.x.append(np.array(vector))
        
        self.n_doc=len(d_line)
        self.n_batch=int(np.ceil(self.n_doc/batch_size))
        rd=random.sample(range(self.n_doc),self.n_doc)
        self.x=np.array(self.x)[rd]
        self.y=np.array(self.y)[rd]
        self.sentence_lengths=np.array(self.sentence_lengths)[rd]
        self.batch_id=0
        self.epoch=0
    
    def next_batch(self):
        self.batch_id+=1
        if(self.batch_id>=self.n_batch-1):
            self.batch_id=0
            self.epoch+=1
            rd=random.sample(range(self.n_doc),self.n_doc)
            self.x=self.x[rd]
            self.y=self.y[rd]
            self.sentence_lengths=self.sentence_lengths[rd]
        
        start=self.batch_id*self.batch_size
        end=start+self.batch_size
        return self.x[start:end],self.y[start:end],self.sentence_lengths[start:end]

In [0]:
import tensorflow.compat.v1 as tf
NUM_CLASSES = 20
MAX_DOC_LENGTH = 500

class RNN:
  def __init__(self, vocab_size, embedding_size, lstm_size, batch_size):
    self._vocab_size = vocab_size
    self._embedding_size = embedding_size
    self._lstm_size = lstm_size
    self._batch_size = batch_size

    self._data = tf.placeholder(tf.int32, shape=[batch_size, MAX_DOC_LENGTH])
    self._labels = tf.placeholder(tf.int32, shape=[batch_size,])
    self._sentence_len = tf.placeholder(tf.int32, shape=[batch_size,])

  def embedding_layer(self, indices):
    pretrain_vecs = []
    pretrain_vecs.append(np.zeros(self._embedding_size))
    np.random.seed(2020)
    for i in range(self._vocab_size+1):
      pretrain_vecs.append(np.random.normal(
          loc=0.,
          scale=1.,
          size=self._embedding_size
      ))
    pretrain_vecs = np.array(pretrain_vecs)
    self._embedding_maxtrix = tf.get_variable(
        name = 'embedding',
        shape = (self._vocab_size+2, self._embedding_size),
        initializer = tf.constant_initializer(pretrain_vecs)
    )

    return tf.nn.embedding_lookup(self._embedding_maxtrix, indices)
  
  def LSTM_layer(self, embeddings):
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(self._lstm_size)
    zero_state = tf.zeros(shape=(self._batch_size, self._lstm_size))
    initial_state = tf.nn.rnn_cell.LSTMStateTuple(zero_state, zero_state)

    lstm_inputs = tf.unstack(tf.transpose(embeddings, perm=[1,0,2]))
    lstm_outputs, last_state = tf.nn.static_rnn(
        cell = lstm_cell,
        inputs = lstm_inputs,
        initial_state = initial_state,
        sequence_length = self._sentence_len
    )
    lstm_outputs = tf.unstack(tf.transpose(lstm_outputs, perm=[1,0,2]))
    lstm_outputs = tf.concat(lstm_outputs, axis=0)

    mask = tf.sequence_mask(
        lengths = self._sentence_len,
        maxlen = MAX_DOC_LENGTH,
        dtype=tf.float32
    )
    mask = tf.concat(tf.unstack(mask, axis=0), axis=0)
    mask = tf.expand_dims(mask, -1)

    lstm_outputs = mask*lstm_outputs
    lstm_outputs_split = tf.split(lstm_outputs, num_or_size_splits=self._batch_size)
    lstm_outputs_sum = tf.reduce_sum(lstm_outputs_split, axis=1)
    lstm_outputs_aver = lstm_outputs_sum / tf.expand_dims(tf.cast(self._sentence_len, tf.float32), -1)

    return lstm_outputs_aver

  def build_graph(self):
    embeddings = self.embedding_layer(self._data)
    lstm_outputs = self.LSTM_layer(embeddings)
    weights = tf.get_variable(
        name = 'final_layer_weights',
        shape = (self._lstm_size, NUM_CLASSES),
        initializer = tf.random_normal_initializer(seed=2020)
    )
    biases = tf.get_variable(
        name = 'final_layer_biases',
        shape = (NUM_CLASSES),
        initializer = tf.random_normal_initializer(seed=2020)
    )
    logits = tf.matmul(lstm_outputs, weights) + biases

    label_one_hot = tf.one_hot(
        indices = self._labels,
        depth = NUM_CLASSES,
        dtype = tf.float32
    )

    loss = tf.nn.softmax_cross_entropy_with_logits(
        labels = label_one_hot,
        logits = logits
    )

    loss = tf.reduce_mean(loss)

    probs = tf.nn.softmax(logits)
    pred_labels = tf.argmax(probs, axis=1)
    pred_labels = tf.squeeze(pred_labels)
  
    return pred_labels, loss

  def trainer(self, loss, lr):
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    return train_op


In [0]:
path_train='/content/drive/My Drive/Datasets/trainencode.txt'
path_test='/content/drive/My Drive/Datasets/testencode.txt'
path_vocab='/content/drive/My Drive/Datasets/vocab.txt'

with open(path_vocab,'r') as f:
    vocab_size=len(f.read().splitlines())
train_data_reader=DataReader(path_train,batch_size=50)
test_data_reader=DataReader(path_test,batch_size=50)

In [0]:
def train_and_eval_RNN():
    tf.set_random_seed(2020)    
    rnn = RNN(
        vocab_size=vocab_size,
        embedding_size=300,
        lstm_size=50,
        batch_size=50
    )

    pred_labels, loss = rnn.build_graph()
    op = rnn.trainer(loss=loss, lr=0.01)

    with tf.Session() as sess:
      step, MAX_STEP = 0, 2000
      sess.run(tf.global_variables_initializer())

      while step < MAX_STEP:
        next_train_batch = train_data_reader.next_batch()
        train_data, train_labels, train_sentence_len = next_train_batch
        plabel_eval,loss_eval, _ = sess.run(
            [pred_labels, loss, op],
            feed_dict={
                rnn._data: train_data,
                rnn._labels: train_labels,
                rnn._sentence_len: train_sentence_len
            }
        )
        step += 1
        if step % 50 == 0:
          print("step: " + str(step) + " loss: " + str(loss_eval))
        if train_data_reader.batch_id == 0:
          num_pre_true=0
          while True:
            next_test_batch = test_data_reader.next_batch()
            test_data, test_labels, test_sentence_len = next_test_batch
            test_plabel_eval= sess.run(
                pred_labels,
                feed_dict={
                    rnn._data: test_data,
                    rnn._labels: test_labels,
                    rnn._sentence_len: test_sentence_len
                }
            )
            matches = np.equal(test_plabel_eval, test_labels)
            num_pre_true += np.sum(matches.astype(float))
            if test_data_reader.batch_id == 0:
                break
          print("Epoch: ", train_data_reader.epoch)
          print("Test accuracy: ", num_pre_true / len(test_data_reader.y))

In [6]:
tf.reset_default_graph()
train_and_eval_RNN()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step: 50 loss: 1.3745732
step: 100 loss: 0.8788212
step: 150 loss: 0.36795798
step: 200 loss: 0.4364363
Epoch:  1
Test accuracy:  0.7448220924057355
step: 250 loss: 0.22916569
step: 300 loss: 0.2664759
step: 350 loss: 0.21291381
step: 400 loss: 0.09855297
step: 450 loss: 0.20761845
Epo